## Installing the dependencies

In [1]:
!!pip -q install langchain
!pip -q install openai
!pip -q install tiktoken
!pip -q install faiss-cpu
!pip -q install langchain_experimental
!pip -q install openai
!pip -q install PyPDF2
!pip -q install templates

In [11]:
pip install pycryptodome

  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/a7/88/5e83de10450027c96c79dc65ac45e9d0d7a7fef334f39d3789a191f33602/pycryptodome-3.21.0-cp36-abi3-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


## Importing the libraries

In [26]:
import openai

openai.api_key = 'MYAPIKEY'


In [20]:
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

## Defining Functions

In [33]:
def get_text_from_pdf(pdf_path):
    """Reads and extracts text from a single PDF file."""
    text = ""
    try:
        file = PdfReader(pdf_path)
        for page in file.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    return text


def chunk_text(raw_text):
    """Splits raw text into smaller chunks for better processing."""
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=900,
        chunk_overlap=300,
        length_function=len
    )
    chunks = text_splitter.split_text(raw_text)
    return chunks


def get_vectorstore(chunks):
    """Creates a FAISS vector store from the text chunks."""
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
    vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)
    return vectorstore


def get_convo_chain(vectorstore):
    """Initializes a conversational retrieval chain with memory."""
    llm = ChatOpenAI(openai_api_key=openai.api_key)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    convo_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return convo_chain


def chat_user_input(user_query, convo_chain):
    """Handles user input and returns the chatbot's response."""
    response = convo_chain({'question': user_query})
    chat_history = response['chat_history']
    return chat_history[-1].content

## Chatbot

In [34]:
def main():
    load_dotenv()
    
    # Define the path for the PDF file
    pdf_path = os.path.join("SJSUReport", "DonationsGuidelines.pdf")
    
    # Check if the PDF file exists
    if not os.path.isfile(pdf_path):
        print(f"Error: The file {pdf_path} does not exist. Please check the file path.")
        return

    print(f"Reading PDF from: {os.path.abspath(pdf_path)}")

    # Extract text from the PDF
    pdf_content = get_text_from_pdf(pdf_path)
    if not pdf_content.strip():
        print("No text was extracted from the PDF. Please ensure it contains extractable text.")
        return

    # Split text into chunks
    chunks = chunk_text(pdf_content)

    # Create vector store
    vectorstore = get_vectorstore(chunks)

    # Get conversation chain
    convo_chain = get_convo_chain(vectorstore)

    print("Ask any query regarding your data or enter 'quit' to exit")

    while True:
        user_query = input("You: ")
        if user_query.lower() == "quit":
            break
        response = chat_user_input(user_query, convo_chain)
        print("ChatBot:", response)


if __name__ == "__main__":
    main()

Reading PDF from: /Users/ravjotsingh/Downloads/Fall 2024/DATA 255 Deep Learning/Class Demos/SJSUReport/DonationsGuidelines.pdf


/var/folders/td/v81nyhcx34j6495f4q71kk280000gn/T/ipykernel_50863/3128405017.py:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Ask any query regarding your data or enter 'quit' to exit


You:  what is the pdf about?


/var/folders/td/v81nyhcx34j6495f4q71kk280000gn/T/ipykernel_50863/3128405017.py:48: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = convo_chain({'question': user_query})


...

You:  where this food is used ?


ChatBot: The food from the Spartan Food Pantry at San Jose State University is used to support students in need on campus.


You:  Summarize the chat for today


ChatBot: Today's chat provided information on how to donate to the Spartan Food Pantry at San Jose State University. The pantry accepts monetary donations through their website, encourages organizing food/toiletry drives, and provides a list of specific items needed for donation. They also offer guidelines for donating, including the types of items needed and drop-off locations. Additionally, they mentioned the option of purchasing items from their Amazon Wishlist for bulk donations.


You:  quit
